In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark import SparkContext

In [2]:
import gensim

# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True) 

/databricks/python/local/lib/python2.7/site-packages/smart_open/smart_open_lib.py:566: UserWarning: streaming gzip support unavailable, see https://github.com/RaRe-Technologies/smart_open/issues/189
 warnings.warn('streaming gzip support unavailable, see %s' % _ISSUE_189_URL)

In [3]:
print (len(model['cat'].tolist()))

300

In [4]:
import re
def split_and_merge_vector(words,model):
  words = words.strip()
  word_list = re.split(' |,|/',words)
  word_list  = list(filter(None, word_list))
#   print(word_list)
  vector_lst = []
  count = 0
  for word in word_list:
    try:
      vector = model[word].tolist()
    except KeyError:
      continue
    vector_lst.append(vector)
    count = count+1
  if count == 0:
    return []
  return [sum(x)/count for x in zip(*vector_lst)]
# split_and_merge_vector(" cat catB cat ",model)

In [5]:
crimeDF = spark.read.format('csv').options(header='true', inferSchema='true').load('/FileStore/tables/Police_Department_Incident_Reports__Historical_2003_to_May_2018.csv').cache()

In [6]:
crimeDF_new = crimeDF.sort("Category").select('Category','Descript')

In [7]:
Category_count = crimeDF_new.groupBy("Category").count()
Category_count = Category_count.sort('count', ascending=False).cache()
Important_Categories = Category_count.filter("`count` >= 100000").cache()

In [8]:
Important_Categories.show()

+--------------+------+
 Category| count|
+--------------+------+
 LARCENY/THEFT|480448|
OTHER OFFENSES|309358|
 NON-CRIMINAL|238323|
 ASSAULT|194694|
 VEHICLE THEFT|126602|
 DRUG/NARCOTIC|119628|
 VANDALISM|116059|
 WARRANTS|101379|
+--------------+------+

In [9]:
import unicodedata
def unicodeToString(uw):
  return unicodedata.normalize('NFKD', uw).encode('ascii','ignore')

In [10]:
def category_des_lst(category):
  cate_df = crimeDF_new.filter(crimeDF_new.Category == category).sort('Descript')
  cate_df = assault_df.distinct()
  # print(assault_df.count())
  # display(assault_df)
  des_assault = assault_df.select('Descript')
  return [unicodeToString(i.Descript) for i in des_assault.collect()]
#example using category assualt to ge list of descriptions
des_assault_lst = category_des_lst('ASSAULT')

In [11]:
def featureVectorForCategory(des_lst):
  feature_vectors = []
  for words in des_lst:
    feature_vectors.append(split_and_merge_vector(words,model))
  return feature_vectors
#example feature map of ASSAULT category
feature_vecs_assualt = featureVectorForCategory(des_assault_lst)

In [12]:
from sklearn.cluster import KMeans
import numpy as np
def Kmean_cluster(num_clusters,word_vectors):
  X = np.array(word_vectors)
  kmeans_fit = KMeans(n_clusters=num_clusters, random_state=0).fit(X)
  cluster_labels = kmeans_fit.labels_
  return cluster_labels
def mergeLabelWithDescription(labels,des_lst):
  tup_lst = zip(labels,des_lst)
  rdd = sc.parallelize(tup_lst)
  return rdd.sortBy(lambda tup: tup[0])
  

In [13]:
#example: display the subcategory with assault
assault_labels = Kmean_cluster(4,feature_vecs_assualt)
assault_rdd = mergeLabelWithDescription(assault_labels,des_assault_lst)
assault_rdd.groupByKey().mapValues(list).collect()

Out[ 180 ]: 
[(0,
 ['AGGRAVATED ASSAULT OF POLICE OFFICER, SNIPING',
 'AGGRAVATED ASSAULT OF POLICE OFFICER,BODILY FORCE',
 'AGGRAVATED ASSAULT ON POLICE OFFICER WITH A GUN',
 'AGGRAVATED ASSAULT ON POLICE OFFICER WITH A KNIFE',
 'AGGRAVATED ASSAULT WITH A DEADLY WEAPON',
 'AGGRAVATED ASSAULT WITH A GUN',
 'AGGRAVATED ASSAULT WITH A KNIFE',
 'AGGRAVATED ASSAULT WITH BODILY FORCE',
 'ASSAULT ON A POLICE OFFICER WITH A DEADLY WEAPON',
 "ASSAULT OR ATTEMPTED MURDER UPON GOV'T OFFICERS",
 'ASSAULT, AGGRAVATED, ON POLICE OFFICER, W/ FULL AUTO',
 'ASSAULT, AGGRAVATED, ON POLICE OFFICER, W/ GUN',
 'ASSAULT, AGGRAVATED, ON POLICE OFFICER, W/ SEMI AUTO',
 'ASSAULT, AGGRAVATED, W/ GUN',
 'ASSAULT, AGGRAVATED, W/ MACHINE GUN',
 'ASSAULT, AGGRAVATED, W/ SEMI AUTO',
 'ATTEMPTED HOMICIDE WITH A DANGEROUS WEAPON',
 'ATTEMPTED HOMICIDE WITH A GUN',
 'ATTEMPTED HOMICIDE WITH A KNIFE',
 'ATTEMPTED HOMICIDE WITH BODILY FORCE',
 'ATTEMPTED HOMICIDE WITH EXPLOSIVES',
 'ATTEMPTED MAYHEM WITH A DEADLY WEAPON',
 'ATTEMPTED MAYHEM WITH A GUN',
 'ATTEMPTED MAYHEM WITH A KNIFE',
 'ATTEMPTED MAYHEM WITH BODILY FORCE',
 'BATTERY OF A POLICE OFFICER',
 'MAYHEM WITH A DEADLY WEAPON',
 'MAYHEM WITH A GUN',
 'MAYHEM WITH A KNIFE',
 'MAYHEM WITH BODILY FORCE']),
 (1,
 ['BATTERY DURING LABOR DISPUTE',
 'BATTERY, FORMER SPOUSE OR DATING RELATIONSHIP',
 'CHILD ABUSE (PHYSICAL)',
 'CHILD, INFLICTING INJURY RESULTING IN TRAUMATIC CONDITION',
 'CIVIL RIGHTS, INCL. INJURY, THREAT, OR DAMAGE (HATE CRIMES)',
 'DISCHARGING IN GROSSLY NEGLIGENT MANNER',
 'ELDER ADULT OR DEPENDENT ABUSE (NOT EMBEZZLEMENT OR THEFT)',
 'FALSE IMPRISONMENT',
 'FIREARM, DISCHARGING AT OCCUPIED BLDG, VEHICLE, OR AIRCRAFT',
 'FIREARM, DISCHARGING IN GROSSLY NEGLIGENT MANNER',
 'LASERS, DISCHARGING OR LIGHTS AT AIRCRAFT',
 'RESISTING PEACE OFFICER, CAUSING THEIR SERIOUS INJURY OR DEATH',
 'SHOOTING INTO INHABITED DWELLING OR OCCUPIED VEHICLE',
 'STALKING',
 'THREAT OR FORCE TO RESIST EXECUTIVE OFFICER',
 'THREAT TO STATE OFFICIAL OR JUDGE',
 'THREATENING PHONE CALL(S)',
 'THREATENING SCHOOL OR PUBLIC EMPLOYEE',
 'THREATS AGAINST LIFE',
 'THREATS TO SCHOOL TEACHERS',
 'TRESPASS WITHIN 30 DAYS OF CREDIBLE THREAT',
 'UNLAWFUL DISSUADING/THREATENING OF A WITNESS',
 'WILLFUL CRUELTY TO CHILD']),
 (2, ['ASSAULT', 'BATTERY']),
 (3,
 ['ASSAULT BY POISONING',
 'ASSAULT BY POLICE OFFICER',
 'ASSAULT WITH CAUSTIC CHEMICALS',
 'ATTEMPTED HOMICIDE BY SNIPING',
 'ATTEMPTED SIMPLE ASSAULT',
 'BATTERY WITH SERIOUS INJURIES',
 'INFLICT INJURY ON COHABITEE',
 'TERRORIZING BY ARSON OR EXPLOSIVE DEVICE',
 'TERRORIZING BY MARKING PRIVATE PROPERTY'])]

In [14]:
import re
def flatternAndCreateDict(des_lst):
  s = set()
  for words in des_lst:
    ws = re.split(' |,|/',words)
    for word in ws:
      s.add(word)
  return list(s)
#example 
unique_word_lst = flatternAndCreateDict(des_assault_lst)
len(unique_word_lst)
print(unique_word_lst)

['', 'MARKING', 'THREATS', 'SIMPLE', 'WITHIN', 'LIFE', 'LABOR', 'AGGRAVATED', 'DURING', 'WITH', 'BY', 'HOMICIDE', 'FORCE', 'WEAPON', 'OFFICIAL', 'OFFICERS', 'CHEMICALS', '(HATE', 'EMPLOYEE', 'DATING', 'CONDITION', 'FULL', 'DWELLING', 'ON', 'PEACE', 'TRESPASS', 'EMBEZZLEMENT', 'SCHOOL', 'OF', 'COHABITEE', 'UNLAWFUL', 'DAMAGE', 'ASSAULT', 'ABUSE', 'JUDGE', 'INJURY', 'OR', 'RESISTING', 'UPON', 'GUN', 'RESIST', 'POISONING', "GOV'T", 'INJURIES', '(PHYSICAL)', 'DISCHARGING', 'EXPLOSIVE', 'TRAUMATIC', 'DEPENDENT', 'DEADLY', 'PHONE', 'THEFT)', 'W', '(NOT', 'WILLFUL', 'DISPUTE', 'PROPERTY', 'TERRORIZING', 'THEIR', 'ATTEMPTED', '30', 'DAYS', 'PUBLIC', 'LIGHTS', 'CREDIBLE', 'INCL.', 'CRIMES)', 'VEHICLE', 'CRUELTY', 'DEATH', 'THREAT', 'SPOUSE', 'FORMER', 'EXPLOSIVES', 'EXECUTIVE', 'TEACHERS', 'INHABITED', 'MURDER', 'DANGEROUS', 'THREATENING', 'BLDG', 'BODILY', 'STATE', 'OFFICER', 'KNIFE', 'FALSE', 'RIGHTS', 'STALKING', 'NEGLIGENT', 'INFLICT', 'FIREARM', 'RELATIONSHIP', 'INFLICTING', 'AIRCRAFT', 'DISSUADING', 'RESULTING', 'LASERS', 'POLICE', 'SEMI', 'ELDER', 'OCCUPIED', 'PRIVATE', 'TO', 'GROSSLY', 'CAUSTIC', 'WITNESS', 'A', 'ARSON', 'AUTO', 'INTO', 'IMPRISONMENT', 'AGAINST', 'MACHINE', 'MAYHEM', 'CAUSING', 'AT', 'IN', 'DEVICE', 'SERIOUS', 'SHOOTING', 'CALL(S)', 'BATTERY', 'CIVIL', 'SNIPING', 'ADULT', 'CHILD', 'MANNER']

In [15]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

In [16]:
def removeStopWords(unique_word_lst):
  stop_words = set(stopwords.words('english')) 
  filtered_sentence = [] 
  for w in unique_word_lst: 
      w_lower = w.lower()
      if w_lower not in stop_words: 
          filtered_sentence.append(w) 
  return filtered_sentence
# print(removeStopWords(['W']))
unique_word_lst = removeStopWords(unique_word_lst)
print(len(unique_word_lst))

114

In [17]:
def createDictVector(word_list,unique_word_lst):
  vector = [0] * len(unique_word_lst)
  for word in word_list:
    indx = unique_word_lst.index(word)
    vector[indx] = 2
  return vector

In [18]:
import re
def split_and_merge_vector2(words,model,unique_word_lst):
  words = words.strip()
  word_list = re.split(' |,|/',words)
  word_list  = list(filter(None, word_list))
  filtered_word_list = removeStopWords(word_list)
  print(filtered_word_list)
#   print(word_list)
  vector_lst = []
  count = 0
  for word in word_list:
    try:
      vector = model[word].tolist()
      vector = vector + createDictVector(filtered_word_list,unique_word_lst)
    except KeyError:
      continue
    vector_lst.append(vector)
    count = count+1
  if count == 0:
    return []
  return [sum(x)/count for x in zip(*vector_lst)]
# split_and_merge_vector2(" cat catB cat ",model,["cat","catB","dog"])[302]

In [19]:
def featureVectorForCategory(des_lst,unique_word_lst):
  feature_vectors = []
  for words in des_lst:
    feature_vectors.append(split_and_merge_vector2(words,model,unique_word_lst))
  return feature_vectors
#example feature map of ASSAULT category
feature_vecs_assualt2 = featureVectorForCategory(des_assault_lst,unique_word_lst)
print(len(feature_vecs_assualt2[0]))

['AGGRAVATED', 'ASSAULT', 'POLICE', 'OFFICER', 'SNIPING']
['AGGRAVATED', 'ASSAULT', 'POLICE', 'OFFICER', 'BODILY', 'FORCE']
['AGGRAVATED', 'ASSAULT', 'POLICE', 'OFFICER', 'GUN']
['AGGRAVATED', 'ASSAULT', 'POLICE', 'OFFICER', 'KNIFE']
['AGGRAVATED', 'ASSAULT', 'DEADLY', 'WEAPON']
['AGGRAVATED', 'ASSAULT', 'GUN']
['AGGRAVATED', 'ASSAULT', 'KNIFE']
['AGGRAVATED', 'ASSAULT', 'BODILY', 'FORCE']
['ASSAULT']
['ASSAULT', 'POISONING']
['ASSAULT', 'POLICE', 'OFFICER']
['ASSAULT', 'POLICE', 'OFFICER', 'DEADLY', 'WEAPON']
['ASSAULT', 'ATTEMPTED', 'MURDER', 'UPON', "GOV'T", 'OFFICERS']
['ASSAULT', 'CAUSTIC', 'CHEMICALS']
['ASSAULT', 'AGGRAVATED', 'POLICE', 'OFFICER', 'W', 'FULL', 'AUTO']
['ASSAULT', 'AGGRAVATED', 'POLICE', 'OFFICER', 'W', 'GUN']
['ASSAULT', 'AGGRAVATED', 'POLICE', 'OFFICER', 'W', 'SEMI', 'AUTO']
['ASSAULT', 'AGGRAVATED', 'W', 'GUN']
['ASSAULT', 'AGGRAVATED', 'W', 'MACHINE', 'GUN']
['ASSAULT', 'AGGRAVATED', 'W', 'SEMI', 'AUTO']
['ATTEMPTED', 'HOMICIDE', 'SNIPING']
['ATTEMPTED', 'HOMICIDE', 'DANGEROUS', 'WEAPON']
['ATTEMPTED', 'HOMICIDE', 'GUN']
['ATTEMPTED', 'HOMICIDE', 'KNIFE']
['ATTEMPTED', 'HOMICIDE', 'BODILY', 'FORCE']
['ATTEMPTED', 'HOMICIDE', 'EXPLOSIVES']
['ATTEMPTED', 'MAYHEM', 'DEADLY', 'WEAPON']
['ATTEMPTED', 'MAYHEM', 'GUN']
['ATTEMPTED', 'MAYHEM', 'KNIFE']
['ATTEMPTED', 'MAYHEM', 'BODILY', 'FORCE']
['ATTEMPTED', 'SIMPLE', 'ASSAULT']
['BATTERY']
['BATTERY', 'LABOR', 'DISPUTE']
['BATTERY', 'POLICE', 'OFFICER']
['BATTERY', 'SERIOUS', 'INJURIES']
['BATTERY', 'FORMER', 'SPOUSE', 'DATING', 'RELATIONSHIP']
['CHILD', 'ABUSE', '(PHYSICAL)']
['CHILD', 'INFLICTING', 'INJURY', 'RESULTING', 'TRAUMATIC', 'CONDITION']
['CIVIL', 'RIGHTS', 'INCL.', 'INJURY', 'THREAT', 'DAMAGE', '(HATE', 'CRIMES)']
['DISCHARGING', 'GROSSLY', 'NEGLIGENT', 'MANNER']
['ELDER', 'ADULT', 'DEPENDENT', 'ABUSE', '(NOT', 'EMBEZZLEMENT', 'THEFT)']
['FALSE', 'IMPRISONMENT']
['FIREARM', 'DISCHARGING', 'OCCUPIED', 'BLDG', 'VEHICLE', 'AIRCRAFT']
['FIREARM', 'DISCHARGING', 'GROSSLY', 'NEGLIGENT', 'MANNER']
['INFLICT', 'INJURY', 'COHABITEE']
['LASERS', 'DISCHARGING', 'LIGHTS', 'AIRCRAFT']
['MAYHEM', 'DEADLY', 'WEAPON']
['MAYHEM', 'GUN']
['MAYHEM', 'KNIFE']
['MAYHEM', 'BODILY', 'FORCE']
['RESISTING', 'PEACE', 'OFFICER', 'CAUSING', 'SERIOUS', 'INJURY', 'DEATH']
['SHOOTING', 'INHABITED', 'DWELLING', 'OCCUPIED', 'VEHICLE']
['STALKING']
['TERRORIZING', 'ARSON', 'EXPLOSIVE', 'DEVICE']
['TERRORIZING', 'MARKING', 'PRIVATE', 'PROPERTY']
['THREAT', 'FORCE', 'RESIST', 'EXECUTIVE', 'OFFICER']
['THREAT', 'STATE', 'OFFICIAL', 'JUDGE']
['THREATENING', 'PHONE', 'CALL(S)']
['THREATENING', 'SCHOOL', 'PUBLIC', 'EMPLOYEE']
['THREATS', 'LIFE']
['THREATS', 'SCHOOL', 'TEACHERS']
['TRESPASS', 'WITHIN', '30', 'DAYS', 'CREDIBLE', 'THREAT']
['UNLAWFUL', 'DISSUADING', 'THREATENING', 'WITNESS']
['WILLFUL', 'CRUELTY', 'CHILD']
414

In [20]:
#example: display the subcategory with assault
assault_labels = Kmean_cluster(5,feature_vecs_assualt2)
assault_rdd = mergeLabelWithDescription(assault_labels,des_assault_lst)
assault_rdd.groupByKey().mapValues(list).collect()

Out[ 342 ]: 
[(0,
 ['BATTERY',
 'BATTERY DURING LABOR DISPUTE',
 'BATTERY WITH SERIOUS INJURIES',
 'BATTERY, FORMER SPOUSE OR DATING RELATIONSHIP']),
 (1,
 ['CHILD ABUSE (PHYSICAL)',
 'CHILD, INFLICTING INJURY RESULTING IN TRAUMATIC CONDITION',
 'CIVIL RIGHTS, INCL. INJURY, THREAT, OR DAMAGE (HATE CRIMES)',
 'DISCHARGING IN GROSSLY NEGLIGENT MANNER',
 'ELDER ADULT OR DEPENDENT ABUSE (NOT EMBEZZLEMENT OR THEFT)',
 'FALSE IMPRISONMENT',
 'FIREARM, DISCHARGING AT OCCUPIED BLDG, VEHICLE, OR AIRCRAFT',
 'FIREARM, DISCHARGING IN GROSSLY NEGLIGENT MANNER',
 'INFLICT INJURY ON COHABITEE',
 'LASERS, DISCHARGING OR LIGHTS AT AIRCRAFT',
 'RESISTING PEACE OFFICER, CAUSING THEIR SERIOUS INJURY OR DEATH',
 'SHOOTING INTO INHABITED DWELLING OR OCCUPIED VEHICLE',
 'STALKING',
 'TERRORIZING BY ARSON OR EXPLOSIVE DEVICE',
 'TERRORIZING BY MARKING PRIVATE PROPERTY',
 'THREAT OR FORCE TO RESIST EXECUTIVE OFFICER',
 'THREAT TO STATE OFFICIAL OR JUDGE',
 'THREATENING PHONE CALL(S)',
 'THREATENING SCHOOL OR PUBLIC EMPLOYEE',
 'THREATS AGAINST LIFE',
 'THREATS TO SCHOOL TEACHERS',
 'TRESPASS WITHIN 30 DAYS OF CREDIBLE THREAT',
 'UNLAWFUL DISSUADING/THREATENING OF A WITNESS',
 'WILLFUL CRUELTY TO CHILD']),
 (2,
 ['AGGRAVATED ASSAULT WITH A DEADLY WEAPON',
 'AGGRAVATED ASSAULT WITH A GUN',
 'AGGRAVATED ASSAULT WITH A KNIFE',
 'AGGRAVATED ASSAULT WITH BODILY FORCE',
 'ASSAULT',
 'ASSAULT BY POISONING',
 'ASSAULT WITH CAUSTIC CHEMICALS',
 'ASSAULT, AGGRAVATED, W/ GUN',
 'ASSAULT, AGGRAVATED, W/ MACHINE GUN',
 'ASSAULT, AGGRAVATED, W/ SEMI AUTO',
 'ATTEMPTED SIMPLE ASSAULT']),
 (3,
 ['AGGRAVATED ASSAULT OF POLICE OFFICER, SNIPING',
 'AGGRAVATED ASSAULT OF POLICE OFFICER,BODILY FORCE',
 'AGGRAVATED ASSAULT ON POLICE OFFICER WITH A GUN',
 'AGGRAVATED ASSAULT ON POLICE OFFICER WITH A KNIFE',
 'ASSAULT BY POLICE OFFICER',
 'ASSAULT ON A POLICE OFFICER WITH A DEADLY WEAPON',
 'ASSAULT, AGGRAVATED, ON POLICE OFFICER, W/ FULL AUTO',
 'ASSAULT, AGGRAVATED, ON POLICE OFFICER, W/ GUN',
 'ASSAULT, AGGRAVATED, ON POLICE OFFICER, W/ SEMI AUTO',
 'BATTERY OF A POLICE OFFICER']),
 (4,
 ["ASSAULT OR ATTEMPTED MURDER UPON GOV'T OFFICERS",
 'ATTEMPTED HOMICIDE BY SNIPING',
 'ATTEMPTED HOMICIDE WITH A DANGEROUS WEAPON',
 'ATTEMPTED HOMICIDE WITH A GUN',
 'ATTEMPTED HOMICIDE WITH A KNIFE',
 'ATTEMPTED HOMICIDE WITH BODILY FORCE',
 'ATTEMPTED HOMICIDE WITH EXPLOSIVES',
 'ATTEMPTED MAYHEM WITH A DEADLY WEAPON',
 'ATTEMPTED MAYHEM WITH A GUN',
 'ATTEMPTED MAYHEM WITH A KNIFE',
 'ATTEMPTED MAYHEM WITH BODILY FORCE',
 'MAYHEM WITH A DEADLY WEAPON',
 'MAYHEM WITH A GUN',
 'MAYHEM WITH A KNIFE',
 'MAYHEM WITH BODILY FORCE'])]